<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/NLP/03_LSTM%26GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#03. LSTM & GRU

Pytorch를 활용한 LSTM 사용법을 학습해보자. 

> Reference: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html


### Part1. LSTM with toy example

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

`torch.nn`을 활용하여 LSTM cell을 생성하는 방법은 다음과 같다. 
* input_size: input x의 feature의 수
* hidden_size: hidden state h의 feature 수
```
lstm = nn.LSTM(input_size, hidden_size)
```

In [ ]:
# input_size:3, hidden_size: 3으로 설정하여 LSTM cell을 설정한다. 
lstm = nn.LSTM(3,3)

LSTM cell을 생성한 후에는 입력으로 들어갈 input x, hidden state h, cell state c를 생성해야 한다. 위에서 정한 input_size와 hidden_size를 고려하여 input과 hidden(h와 c)를 생성하자. 

In [ ]:
# sequence length가 5인 input을 생성한다. 
# 이때, input_size를 3으로 설정했으므로, 3차원 벡터 5개를 생성해야 한다. 
inputs = [torch.randn(1,3) for _ in range(5)]   #torch.randn(a, b)-> a, b는 차원의 개수이다/
#즉, torch.randn(1,3)은 1번째에는 1개를, 2번째에는 3개의 요소를 가진다. 

# lstm은 input x와 hidden state h를 입력으로 받으므로 hidden state도 생성한다.
# 이때, hidden_size를 3으로 설정했으므로, 3차원 벡터를 생성한다. 
# lstm의 input으로 들어가는 h는 RNN에서의 hidden state와 lstm에서 등장한 개념인 cell state로 구성되므로 
# hidden은 3차원 벡터 2개로 구성되어야 한다. 
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

In [ ]:
inputs

In [ ]:
hidden

방법1: sequence length가 5인 input에 대하여 한 번에 하나의 element를 lstm cell에 통과시킨다. 

In [ ]:
for i in inputs:
    # 한 번에 한 요소씩 시퀀스를 진행한다. 
    # 각 step 후에, hidden은 hidden state를 포함한다. 
    out, hidden  = lstm(i.view(1,1,-1), hidden)   #view:np의 reshape과 같은 기능. 형태 변환.

방법2: 전체 시퀀스를 한번에 통과시키는 방법.

LSTM이 반환하는 출력의 첫번째 값은 전체 시퀀스에 대한 통과한 hidden state이고, 두 번째 값은 마지막 step의 hidden state이다. 

`out`과 `hidden`의 size를 비교하자. 

In [ ]:
# 방법 2 를 적용하기 위해 input 을 list 가 아닌 하나의 tensor 로 concat 해줍니다.
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# 방법 2 를 적용하기 위해 hidden 을 다시 초기화합니다.
hidden = (torch.randn(1,1,3), torch.randn(1,1,3))

out, hidden = lstm(inputs, hidden)

print(out)
print(hidden)

### Part 2. LSTM for Part-of-Speech Tagging

LSTM을 이용해 Part-of-Speech(POS) Tagging을 하기 위해 학습 데이터를 준비한다. 

* `Training_data`에는 단어 시퀀스와 각 단어의 품사 태그를 준비해야 한다. 
* `word_to_ix`: 모델의 input으로 사용하기 위해 각 단어를 id로 mapping한다. 
* `tag_to_ix`: 품사 태그 또한 id로 mapping한다. 

In [ ]:
#
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype = torch.long)

In [ ]:
training_data=[
    # Tags: DET: 한정사, NN: 명사, V: 동사
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

#모델의 input을 만들기 위해 각 word를 id로 mapping한다. 
word_to_ix = {}   #mapping할 단어들의 모음

#각 시퀀스와 태그 리스트를 튜플에 넣는다. 
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:                 # word가 index를 아직 가지고 있지 않다면
            word_to_ix[word] = len(word_to_ix)     # unique index를 할당해준다. 

print(word_to_ix)

#unique index마다 태그를 할당한다. 
tag_to_ix = {"DET":0, "NN":1, "V":2}


# 보통은 32 or 64 차원에 가깝고 
# 이 차원을 작게 유지하면서 가중치가 어떻게 변하는지 확인한다. 
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

Embedding layer, output layer, lstm cell을 포함한 LSTM Tagger 모듈을 정의하자. 

* `embeds`: input id를 embedding layer로 encode하여 input에 해당하는 embedding을 생성한다. 
* `lstm_out`: embedding을 lstm에 통과하여 전체 시퀀스에 대한 hidden state를 저장한다. 
* `tag_space`: lstm의 output인 hidden을 이용해 존재하는 tag(DET, NN, V) 공간으로 linear transform한다. 
* `tag_scores`: 이후 softmax를 적용하여 각 tag가 될 score를 측정한다. 

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM은 input으로 word embedding을 가지고 output으로 hidden state를 가진다. 
        # 이것들은 hidden_dim을 차원의 수로 가진다? 
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        #linear layer는 hidden state space에서 tag space으로 매핑시킨다. 
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

정의한 LSTM Tagger 클래스를 이용해 model을 build 하고, 학습에 필요한 loss함수와 optimizer를 선언한다. 


In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

모델을 학습하기 전 tag score를 확인하자.

In [ ]:
# 학습전에 score를 확인하자.
# word i에 대한 tag의 스코어를 j로 하는 output을 print한다. 
# 여기서는 학습을 진행하지 않으므로 torch.no_grad()로 wrap한다. 
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

이제, training data를 이용해 모델을 학습한다. input을 LSTMTagger에 통과시켜 각 단어의 Pos tag를 예측하고 정답 tag와 비교하여 loss를 계산한 후 loss를 backpropagate하여 업데이트한다. 

In [ ]:

for epoch in range(300):  
    for sentence, tags in training_data:
        # Step 1. pytorch는 gradient를 축적한다. 
        # 그러므로 반복 전에 정리해야 한다. 
        model.zero_grad()

        # Step 2. 네트워크에 대한 입력을 준비한다.
        # tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. 실행시킨다.
        tag_scores = model(sentence_in)

        # Step 4. loss, gradients를 계산하고 parameters를 업데이트한다.
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

학습 데이터를 이용해 모델이 잘 학습되었는지 확인하자. 학습하기 전 tag score와 비교해보자. 

In [ ]:
# 학습 후 training score를 확인하자. 
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)